In [1]:
import cv2
from imutils import contours
import numpy as np


class Detector:
    def __init__(self, verbose=False, resize=True, width=400, height=400):
        self.verbose = verbose
        self.resize = resize
        self.width = width
        self.height = height

    def detect(self, image_path):
        # read image from path
        image = cv2.imread(image_path)
        if self.resize:
            height, width, _ = image.shape
            image = cv2.resize(image, (min(self.width, width), min(self.height, height)),
                               interpolation=cv2.INTER_AREA)

        # turn image into gray-scale
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # adaptive thresholding
        image_proc = cv2.adaptiveThreshold(image_gray, 255,
                                           cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 20)

        # image dilation
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        image_proc = cv2.dilate(image_proc, rect_kernel, iterations=1)

        # noise removal with opening(erode + dilate)
        # noise_kernel = np.ones((2, 2), np.uint8)
        # image_proc = cv2.morphologyEx(image_proc, cv2.MORPH_OPEN, noise_kernel)

        # find contours
        cnts, hierarchy = cv2.findContours(image_proc, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts, _ = contours.sort_contours(cnts, method="left-to-right")

        # save contours anc coordinates in lists
        crop_list = list()
        crop_coord = list()
        for c in cnts:
            if cv2.contourArea(c) > 40:
                x, y, w, h = cv2.boundingRect(c)
                crop = 255 - image[y:y + h, x:x + w]
                crop = cv2.resize(crop, (30, 30), interpolation=cv2.INTER_AREA)
                crop_list.append(crop)
                crop_coord.append([x, y, w, h])
                if self.verbose:
                    cv2.rectangle(image, (x, y), (x + w, y + h), (36, 255, 12), 2)

        if self.verbose:
            cv2.imshow("Threshold", image_proc)
            cv2.imshow("Image", image)
            cv2.waitKey(0)
        return np.array(crop_list), np.array(crop_coord)


In [2]:
import numpy as np
import os
import cv2
import pickle

IMG_WIDTH = 30
IMG_HEIGHT = 30


def create_dataset(img_folder):
    for folder in os.listdir(img_folder):
        img_data = []
        if folder == "results":
            continue
        print(folder)
        for file in os.listdir(os.path.join(img_folder, folder)):
            image_path = os.path.join(img_folder, folder, file)
            image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_AREA)
            image = cv2.bitwise_not(image)
            image = np.array(image)
            image = image.astype('float32')
            image /= 255
            img_data.append([image, folder])
        with open("./dataset/results/" + folder + ".pickle", "wb") as pickle_file:
            pickle.dump(img_data, pickle_file)


def load_dataset(dataset_folder):
    dataset = []
    for file in os.listdir(dataset_folder):
        with open(dataset_folder + file, "rb") as pickle_file:
            data = pickle.load(pickle_file)
            dataset.extend(data)

    dataset = np.array(dataset)
    X, y = dataset[:, 0], dataset[:, 1]

    Xn = np.zeros(shape=(len(X), len(X[0]), len(X[0][0]), 1))
    for i in range(len(X)):
        Xn[i] = np.expand_dims(X[i], axis=(0, 3))
    return Xn, y


def encode_labels(labels):
    encoder = {
        "0": 0,
        "1": 1,
        "2": 2,
        "3": 3,
        "4": 4,
        "5": 5,
        "6": 6,
        "7": 7,
        "8": 8,
        "9": 9,
        "+": 10,
        "-": 11,
        "div": 12,
        "times": 13,
        "(": 14,
        ")": 15
    }
    res = np.zeros(len(labels))
    for i in range(len(labels)):
        res[i] = encoder.get(labels[i])
    return res


In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import Conv2D
from detector import Detector
from dataset import create_dataset, load_dataset, encode_labels
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, MaxPool2D, Flatten
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from tqdm.keras import TqdmCallback
import numpy as np
import cv2


def compile_model(optimizer="adam", loss="categorical_crossentropy", metrics="accuracy"):
    model = Sequential()

    # first layer - convolution
    model.add(Conv2D(32, 3, activation='relu', input_shape=(30, 30, 1)))

    # second layer - pooling
    model.add(MaxPool2D(pool_size=(2, 2)))

    # dropout
    model.add(Dropout(0.3))

    #
    model.add(Flatten())

    # output
    model.add(Dense(16, activation='softmax'))

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics)
    return model

In [5]:
detector = Detector(verbose=False)
crop_list, crop_coord = detector.detect("./data/test2.jpg")
# cv2.imshow("crop", crop_list[0])
# cv2.waitKey(0)
# create_dataset("./dataset/")

X, y = load_dataset("./dataset/results/")
X = np.asarray(X).astype('float32')
y = encode_labels(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = compile_model()
model.fit(X_train, y_train, epochs=1)
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss)
print(val_acc)

/home/ikrizanic/Documents/git_repos/photomath_task/dataset.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dataset = np.array(dataset)


1746/1746 [==============================] - 46s 26ms/step - loss: 0.1544 - accuracy: 0.9579

In [ ]:
crop_list, crop_coord = detector.detect("./data/test3.jpg")
predicitons = []
for i, crop in enumerate(crop_list):
    image = np.array(crop)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # image = cv2.bitwise_not(image)
    image = image.astype('float32')
    image /= 255
    predicitons.append(model.predict(np.expand_dims(image, axis=(0,3))))

In [1]:
import matplotlib.pyplot as plt
test = X[1000]
plt.imshow("test", test)

NameError: name 'X' is not defined

In [ ]:
for p in predicitons:
    print(np.argmax(p))